# Environment

In [2]:
import requests
import json
import time
import pprint

headers = {
    'Content-Type': 'application/json'
}

HOST='localhost'
URL_PREFIX = 'http://{host}:9200'.format(host=HOST)

INDEX_NAME_CONDUCTOR_WORKFLOW="workflow"
INDEX_NAME_CONDUCTOR_TASK="task"

# Requests
# https://docs.python-requests.org/en/latest/api
def print_request(pr):
    body = pr.body
    if body != None and len(body) > 1000:
        body = str(body[:100]) + '...'
    print("""=== >
{method} {url}
{headers}
{body}""".format(method=pr.method, url=pr.url, headers=pr.headers, body=body))

def print_response(r):
    print("====== Response\n", r.status_code)
    print_request(r.request)
    print("=== <")
    print(r.headers)
    if 'content-type' in r.headers and  'application/json' in r.headers['content-type']:
        pprint.pprint(r.json())
    else:
        if len(r.content) > 1000:
            print(r.content[:100], '...')
        else:
            print(r.content)

# Getting started

## Priming Elasticsearch with data

In [5]:
datas = [{
    "title": "Effective Java",
    "author": "Joshua Bloch",
    "release_date": "2001-06-01",
    "amazon_rating": 4.7,
    "best_seller": True,
    "prices": {
        "usd": 9.95,
        "gbp": 7.95,
        "eur": 8.95
    }
},
    {
    "title": "Core Java Volume I - Fundamentals",
    "author": "Cay S. Horstmann",
    "release_date": "2018-08-27",
    "amazon_rating": 4.8,
    "best_seller": True,
    "prices": {
        "usd": 19.95,
        "gbp": 17.95,
        "eur": 18.95
    }
},
    {
    "title": "Java: A Beginner's Guide",
    "author": "Herbert Schildt",
    "release_date": "2018-11-20",
    "amazon_rating": 4.2,
    "best_seller": True,
    "prices": {
        "usd": 19.99,
        "gbp": 19.99,
        "eur": 19.99
    }
}]

_idx = 0
for data in datas:
  _idx = _idx + 1
  url = URL_PREFIX + \
      '/{index_name}/_doc/{doc_id}'.format(index_name='books', doc_id=_idx)
  params = {}

  print(url)
  pprint.pprint(headers)
  pprint.pprint(params)

  r = requests.put(url=url, headers=headers,
                  params=params, data=json.dumps(data))
  print_response(r)

http://localhost:9200/books/_doc/1
{'Content-Type': 'application/json'}
{}
====== Response
 201
=== >
PUT http://localhost:9200/books/_doc/1
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '177'}
{"title": "Effective Java", "author": "Joshua Bloch", "release_date": "2001-06-01", "amazon_rating": 4.7, "best_seller": true, "prices": {"usd": 9.95, "gbp": 7.95, "eur": 8.95}}
=== <
{'Location': '/books/_doc/1', 'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '139'}
{'_id': '1',
 '_index': 'books',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': '_doc',
 '_version': 1,
 'result': 'created'}
http://localhost:9200/books/_doc/2
{'Content-Type': 'application/json'}
{}
====== Response
 201
=== >
PUT http://localhost:9200/books/_doc/

## Retrieving data

### _count

In [6]:
url = URL_PREFIX + '/{index_name}/_count'.format(index_name='books')
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:9200/books/_count
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:9200/books/_count
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '89'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 3}


### _doc/{doc_id}

In [7]:
url = URL_PREFIX + '/{index_name}/_doc/{doc_id}'.format(index_name='books', doc_id=1)
params = {
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.get(url=url, headers=headers, params=params)
print_response(r)

http://localhost:9200/books/_doc/1
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
GET http://localhost:9200/books/_doc/1
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json'}
None
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '228'}
{'_id': '1',
 '_index': 'books',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'amazon_rating': 4.7,
             'author': 'Joshua Bloch',
             'best_seller': True,
             'prices': {'eur': 8.95, 'gbp': 7.95, 'usd': 9.95},
             'release_date': '2001-06-01',
             'title': 'Effective Java'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


### _search ids

In [8]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "ids": {
            "values": [1, 2, 3]
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '41'}
{"query": {"ids": {"values": [1, 2, 3]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '433'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'books',
                    '_score': 1.0,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'best_seller': True,
                                'prices': {'eur': 8.95,
                                           'gbp': 7.95,
                              

### _search match_all

In [10]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {}
# same as:
# data = {
#     "query": {
#         "match_all": {}
#     }
# }
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '2'}
{}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '431'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'books',
                    '_score': 1.0,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'best_seller': True,
                                'prices': {'eur': 8.95,
                                           'gbp': 7.95,
                                           'usd': 9.95},
             

## Full-text search

### match

In [16]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "match": {
            # "author": "Joshua"
            # "author": "JoShUa"
            # "author":"joshua"
            # "author": "JOSHUA"
            # "author":"Bloch"
            "author": "josh" # match 0
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '40'}
{"query": {"match": {"author": "josh"}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '140'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 2}


### prefix

In [17]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "prefix": {
            "author": "josh"
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '41'}
{"query": {"prefix": {"author": "josh"}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '290'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'books',
                    '_score': 1.0,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'best_seller': True,
                                'prices': {'eur': 8.95,
                                           'gbp': 7.95,
                              

### match operator: AND, OR(default)

In [20]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "match": {
            "author": "Joshua Schildt"  # match 2
        }
    }
}
data = {
    "query": {
        "match": {
            "author": {
                "query": "Joshua Schildt",  # match 0
                "operator": "AND"
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '80'}
{"query": {"match": {"author": {"query": "Joshua Schildt", "operator": "AND"}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '141'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 5}


### _bulk

In [37]:
url = URL_PREFIX + '/_bulk'
params = {
}
data = """{"index":{"_index":"books","_id":"1"}}
{"title": "Core Java Volume I - Fundamentals","author": "Cay S. Horstmann","edition": 11, "synopsis": "Java reference book that offers a detailed explanation of various features of Core Java, including exception handling, interfaces, and lambda expressions. Significant highlights of the book include simple language, conciseness, and detailed examples.","amazon_rating": 4.6,"release_date": "2018-08-27","tags": ["Programming Languages, Java Programming"]}
{"index":{"_index":"books","_id":"2"}}
{"title": "Effective Java","author": "Joshua Bloch", "edition": 3,"synopsis": "A must-have book for every Java programmer and Java aspirant, Effective Java makes up for an excellent complementary read with other Java books or learning material. The book offers 78 best practices to follow for making the code better.", "amazon_rating": 4.7, "release_date": "2017-12-27", "tags": ["Object Oriented Software Design"]}
{"index":{"_index":"books","_id":"3"}}
{"title": "Java: A Beginner's Guide", "author": "Herbert Schildt","edition": 8,"synopsis": "One of the most comprehensive books for learning Java. The book offers several hands-on exercises as well as a quiz section at the end of every chapter to let the readers self-evaluate their learning.","amazon_rating": 4.2,"release_date": "2018-11-20","tags": ["Software Design & Engineering", "Internet & Web"]}
{"index":{"_index":"books","_id":"4"}}
{"title": "Java - The Complete Reference","author": "Herbert Schildt","edition": 11,"synopsis": "Convenient Java reference book examining essential portions of the Java API library, Java. The book is full of discussions and apt examples to better Java learning.","amazon_rating": 4.4,"release_date": "2019-03-19","tags": ["Software Design & Engineering", "Internet & Web", "Computer Programming Language & Tool"]}
{"index":{"_index":"books","_id":"5"}}
{"title": "Head First Java","author": "Kathy Sierra and Bert Bates","edition":2, "synopsis": "The most important selling points of Head First Java is its simplicity and super-effective real-life analogies that pertain to the Java programming concepts.","amazon_rating": 4.3,"release_date": "2005-02-18","tags": ["IT Certification Exams", "Object-Oriented Software Design","Design Pattern Programming"]}
{"index":{"_index":"books","_id":"6"}}
{"title": "Java Concurrency in Practice","author": "Brian Goetz with Tim Peierls, Joshua Bloch, Joseph Bowbeer, David Holmes, and Doug Lea","edition": 1,"synopsis": "Java Concurrency in Practice is one of the best Java programming books to develop a rich understanding of concurrency and multithreading.","amazon_rating": 4.3,"release_date": "2006-05-09","tags": ["Computer Science Books", "Programming Languages", "Java Programming"]}
{"index":{"_index":"books","_id":"7"}}
{"title": "Test-Driven: TDD and Acceptance TDD for Java Developers","author": "Lasse Koskela","edition": 1,"synopsis": "Test-Driven is an excellent book for learning how to write unique automation testing programs. It is a must-have book for those Java developers that prioritize code quality as well as have a knack for writing unit, integration, and automation tests.","amazon_rating": 4.1,"release_date": "2007-10-22","tags": ["Software Architecture", "Software Design & Engineering", "Java Programming"]}
 {"index":{"_index":"books","_id":"8"}}
 {"title": "Head First Object-Oriented Analysis Design","author": "Brett D. McLaughlin, Gary Pollice & David West","edition": 1,"synopsis": "Head First is one of the most beautiful finest book series ever written on Java programming language. Another gem in the series is the Head First Object-Oriented Analysis Design.","amazon_rating": 3.9,"release_date": "2014-04-29","tags": ["Introductory & Beginning Programming", "Object-Oriented Software Design", "Java Programming"]}
 {"index":{"_index":"books","_id":"9"}}
 {"title": "Java Performance: The Definite Guide","author": "Scott Oaks","edition": 1,"synopsis": "Garbage collection, JVM, and performance tuning are some of the most favorable aspects of the Java programming language. It educates readers about maximizing Java threading and synchronization performance features, improve Java-driven database application performance, tackle performance issues","amazon_rating": 4.1,"release_date": "2014-03-04","tags": ["Design Pattern Programming", "Object-Oriented Software Design", "Computer Programming Language & Tool"]}
 {"index":{"_index":"books","_id":"10"}}
 {"title": "Head First Design Patterns", "author": "Eric Freeman & Elisabeth Robson with Kathy Sierra & Bert Bates","edition": 10,"synopsis": "Head First Design Patterns is one of the leading books to build that particular understanding of the Java programming language." ,"amazon_rating": 4.5,"release_date": "2014-03-04","tags": ["Design Pattern Programming", "Object-Oriented Software Design eTextbooks", "Web Development & Design eTextbooks"]}
"""

print(url)
_headers = {
  # see org.elasticsearch.xcontent.XContentType#fromMediaType
  'Content-Type': 'application/x-ndjson'
}
pprint.pprint(_headers)
pprint.pprint(params)

r = requests.post(url=url, headers=_headers, params=params, data=data)
print_response(r)

http://localhost:9200/_bulk
{'Content-Type': 'application/x-ndjson'}
{}
====== Response
 200
=== >
POST http://localhost:9200/_bulk
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/x-ndjson', 'Content-Length': '4919'}
{"index":{"_index":"books","_id":"1"}}
{"title": "Core Java Volume I - Fundamentals","author": "Cay ...
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '276'}
{'errors': False,
 'items': [{'index': {'_id': '1',
                      '_index': 'books',
                      '_primary_term': 1,
                      '_seq_no': 4,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_type': '_doc',
                      '_version': 3,
                      'result': 'updated',
                      'status': 200}},
           {'index': 

### multi_match

In [36]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "multi_match": {
            "query": "Java",
            # "fields": ["title", "synopsis"]
            "fields": ["title^3","synopsis"] # boosting
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '80'}
{"query": {"multi_match": {"query": "Java", "fields": ["title^3", "synopsis"]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '454'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'books',
                    '_score': 0.37510094,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'best_seller': True,
                                'prices': {'eur': 8.95,
                                        

### match_phrase

In [43]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "match_phrase": {
            "synopsis": {
                "query": "must-have book for every Java programmer",
                "slop": 1
            }
        }
    },
    "highlight": {  # highlights
        "fields": {
            "synopsis": {}
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '150'}
{"query": {"match_phrase": {"synopsis": {"query": "must-have book for every Java programmer", "slop": 6}}}, "highlight": {"fields": {"synopsis": {}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '493'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'books',
                    '_score': 7.300332,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'edition': 3,
                                'r

### fuzziness

In [46]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "match": {
            "tags": {
                "query": "Komputer",
                "fuzziness": 1
            }
        }
    },
    "highlight": {  # highlights
        "fields": {
            "synopsis": {}
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '112'}
{"query": {"match": {"tags": {"query": "Komputer", "fuzziness": 1}}}, "highlight": {"fields": {"synopsis": {}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '893'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '6',
                    '_index': 'books',
                    '_score': 1.0458735,
                    '_source': {'amazon_rating': 4.3,
                                'author': 'Brian Goetz with Tim Peierls, '
                                          'Joshua Bloch, Joseph Bowbeer, David '
                   

## Term-level queries

### term

In [47]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "term": {
            "edition": {
                "value": 3
            }
        }
    },
    "_source": ["title", "edition"]
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '79'}
{"query": {"term": {"edition": {"value": 3}}}, "_source": ["title", "edition"]}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '203'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'books',
                    '_score': 1.0,
                    '_source': {'edition': 3, 'title': 'Effective Java'},
                    '_type': '_doc'}],
          'max_score': 1.0,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 30}


### range

In [49]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "range": {
            "amazon_rating": {
                "gte": 4.5,
                "lte": 5
            }
        }
    },
    "_source": ["title", "amazon_rating"]
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '102'}
{"query": {"range": {"amazon_rating": {"gte": 4.5, "lte": 5}}}, "_source": ["title", "amazon_rating"]}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '278'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'books',
                    '_score': 1.0,
                    '_source': {'amazon_rating': 4.6,
                                'title': 'Core Java Volume I - Fundamentals'},
                    '_type': '_doc'},
                   {'_id': '2',
                    '_index': 'books',
   

## Compound queries

### bool must

In [9]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "author": "Joshua Bloch"
                }
            },
                {
                "match_phrase": {
                    "synopsis": "best Java programming books"
                }
            }]
        }
    },
    # "_source": ["title", "author"]
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '133'}
{"query": {"bool": {"must": [{"match": {"author": "Joshua Bloch"}}, {"match_phrase": {"synopsis": "best Java programming books"}}]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '435'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '6',
                    '_index': 'books',
                    '_score': 5.2532625,
                    '_source': {'amazon_rating': 4.3,
                                'author': 'Brian Goetz with Tim Peierls, '
                                          'Joshua Bloch, Joseph Bowbeer, David 

### bool must_not

In [8]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "author": "Joshua Bloch"
                }
            }],
            "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}]
        }
    },
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '127'}
{"query": {"bool": {"must": [{"match": {"author": "Joshua Bloch"}}], "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '439'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'books',
                    '_score': 3.8919764,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'edition': 3,
                                'release_date': '2017-12

### bool should

In [10]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "author": "Joshua Bloch"
                }
            }],
            "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}],
            "should": [{"match": {"tags": "Software"}}]
        }
    },
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '172'}
{"query": {"bool": {"must": [{"match": {"author": "Joshua Bloch"}}], "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}], "should": [{"match": {"tags": "Software"}}]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '438'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'books',
                    '_score': 4.213981,
                    '_source': {'amazon_rating': 4.7,
                                'author': 'Joshua Bloch',
                                'edition': 3,
            

### bool filter

In [13]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "query": {
        "bool": {
            "must": [{
                "match": {
                    "author": "Joshua Bloch"
                }
            }],
            "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}],
            "should": [{"match": {"tags": "Software"}}],
            "filter": [
                {"range": {"release_date": {"gte": "2015-01-01"}}},
                {"term": {"edition": 3}}
            ]
        }
    },
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '262'}
{"query": {"bool": {"must": [{"match": {"author": "Joshua Bloch"}}], "must_not": [{"range": {"amazon_rating": {"lt": 4.7}}}], "should": [{"match": {"tags": "Software"}}], "filter": [{"range": {"release_date": {"gte": "2015-01-01"}}}, {"term": {"edition": 3}}]}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '438'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '2',
                    '_index': 'books',
                    '_score': 4.213981,
                    '_source': {'amazon_rating': 4.7,
                          

## Aggregations

### covid data

In [14]:
url = URL_PREFIX + '/covid/_bulk'
params = {
}
data = """{"index":{}}
{"country":"United States of America","date":"2021-03-26","cases":30853032,"deaths":561142,"recovered":23275268,"critical":8610}
{"index":{}}
{"country":"Brazil","date":"2021-03-26","cases":12407323,"deaths":307326,"recovered":10824095,"critical":8318}
{"index":{}}
{"country":"India","date":"2021-03-26","cases":11908373,"deaths":161275,"recovered":11292849,"critical":8944}
{"index":{}}
{"country":"Russia","date":"2021-03-26","cases":4501859,"deaths":97017,"recovered":4120161,"critical":2300}
{"index":{}}
{"country":"France","date":"2021-03-26","cases":4465956,"deaths":94275,"recovered":288062,"critical":4766}
{"index":{}}
{"country":"United kingdom","date":"2021-03-26","cases":4325315,"deaths":126515,"recovered":3768434,"critical":630}
{"index":{}}
{"country":"Italy","date":"2021-03-26","cases":3488619,"deaths":107256,"recovered":2814652,"critical":3628}
{"index":{}}
{"country":"Spain","date":"2021-03-26","cases":3255324,"deaths":75010,"recovered":3016247,"critical":1830}
{"index":{}}
{"country":"Turkey","date":"2021-03-26","cases":3149094,"deaths":30772,"recovered":2921037,"critical":1810}
{"index":{}}
{"country":"Germany","date":"2021-03-26","cases":2754002,"deaths":76303,"recovered":2467600,"critical":3209}
"""

print(url)
_headers = {
  # see org.elasticsearch.xcontent.XContentType#fromMediaType
  'Content-Type': 'application/x-ndjson'
}
pprint.pprint(_headers)
pprint.pprint(params)

r = requests.post(url=url, headers=_headers, params=params, data=data)
print_response(r)

http://localhost:9200/covid/_bulk
{'Content-Type': 'application/x-ndjson'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_bulk
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/x-ndjson', 'Content-Length': '1243'}
{"index":{}}
{"country":"United States of America","date":"2021-03-26","cases":30853032,"deaths":561...
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '286'}
{'errors': False,
 'items': [{'index': {'_id': 'CC4GwZABImMvVPAdSxVg',
                      '_index': 'covid',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_type': '_doc',
                      '_version': 1,
                      'result': 'created',
                      'status

### aggs avg

In [16]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='books')
params = {
}
data = {
    "size": 0, # no original source documents
    "aggs": {
        "avg_rating": {
            "avg": {
                "field": "amazon_rating"
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/books/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/books/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '72'}
{"size": 0, "aggs": {"avg_rating": {"avg": {"field": "amazon_rating"}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '177'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'avg_rating': {'value': 4.309999990463257}},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 10}},
 'timed_out': False,
 'took': 94}


### aggs sum

In [17]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='covid')
params = {
}
data = {
    "size": 0,  # no original source documents
    "aggs": {
        "critical_patients": {
            "sum": {
                "field": "critical"
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/covid/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '74'}
{"size": 0, "aggs": {"critical_patients": {"sum": {"field": "critical"}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '174'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'critical_patients': {'value': 44045.0}},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 10}},
 'timed_out': False,
 'took': 27}


### aggs max

In [19]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='covid')
params = {
}
data = {
    "size": 0,  # no original source documents
    "aggs": {
        "max_deaths": {
            "max": {
                "field": "deaths"
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/covid/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '65'}
{"size": 0, "aggs": {"max_deaths": {"max": {"field": "deaths"}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '167'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'max_deaths': {'value': 561142.0}},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 10}},
 'timed_out': False,
 'took': 1}


### aggs stats

In [20]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='covid')
params = {
}
data = {
    "size": 0,  # no original source documents
    "aggs": {
        "all_stats": {
            "stats": {
                "field": "deaths"
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/covid/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '66'}
{"size": 0, "aggs": {"all_stats": {"stats": {"field": "deaths"}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '201'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'all_stats': {'avg': 163689.1,
                                'count': 10,
                                'max': 561142.0,
                                'min': 30772.0,
                                'sum': 1636891.0}},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 10}},
 'timed_out': False,
 'took': 18}


### bucket histogram

In [21]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='covid')
params = {
}
data = {
    "size": 0,  # no original source documents
    "aggs": {
        "critical_patients_as_histogram": {
            "histogram": {
                "field": "critical",
                "interval": 2500
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/covid/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '111'}
{"size": 0, "aggs": {"critical_patients_as_histogram": {"histogram": {"field": "critical", "interval": 2500}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '222'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'critical_patients_as_histogram': {'buckets': [{'doc_count': 4,
                                                                  'key': 0.0},
                                                                 {'doc_count': 3,
                                                                  'key': 2500.0},
                 

### bucket range

In [22]:
url = URL_PREFIX + '/{index_name}/_search'.format(index_name='covid')
params = {
}
data = {
    "size": 0,  # no original source documents
    "aggs": {
        "range_countries": {
            "range": {
                "field": "deaths",
                "ranges": [
                    {"to": 60000},
                    {"from": 60000, "to": 70000},
                    {"from": 70000, "to": 80000},
                    {"from": 80000, "to": 120000}
                ]
            }
        }
    }
}
print(url)
pprint.pprint(headers)
pprint.pprint(params)

r = requests.post(url=url, headers=headers,
                  params=params, data=json.dumps(data))
print_response(r)

http://localhost:9200/covid/_search
{'Content-Type': 'application/json'}
{}
====== Response
 200
=== >
POST http://localhost:9200/covid/_search
{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Content-Length': '190'}
{"size": 0, "aggs": {"range_countries": {"range": {"field": "deaths", "ranges": [{"to": 60000}, {"from": 60000, "to": 70000}, {"from": 70000, "to": 80000}, {"from": 80000, "to": 120000}]}}}}
=== <
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'content-length': '252'}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'aggregations': {'range_countries': {'buckets': [{'doc_count': 1,
                                                   'key': '*-60000.0',
                                                   'to': 60000.0},
                                                  {'doc_co